<a href="https://colab.research.google.com/github/DrPepper18/Bluscreendez-website/blob/main/NLP_Disaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import re

In [82]:
df_train = pd.read_csv('train.csv').drop(columns = ['id', 'keyword', 'location'])
df_train.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [83]:
vocab = {
    0: {},
    1: {},
}
train_size = df_train.shape[0]
min_freq = 5
min_word_length = 4

for i in range(train_size):
    text = re.sub('[^a-z]+', ' ', str(df_train['text'][i]).lower()).split()
    for word in text:
        if len(word) >= min_word_length:
            if word in vocab[df_train['target'][i]].keys():
                vocab[df_train['target'][i]][word] += 1
            else:
                vocab[df_train['target'][i]][word] = 1

buf0 = sorted(vocab[0].items(), key = lambda x: x[1])
buf1 = sorted(vocab[1].items(), key = lambda x: x[1])
vocab[0] = {}
vocab[1] = {}

for item in buf0:
    if item[1] >= min_freq:
        vocab[0][item[0]] = item[1]
for item in buf1:
    if item[1] >= min_freq:
        vocab[1][item[0]] = item[1]

print(len(vocab[0]), '\n', vocab[0], '\n', sorted(vocab[0]))
print(len(vocab[1]), '\n', vocab[1], '\n', sorted(vocab[1]))

1437 
 {'wonderful': 5, 'doubt': 5, 'father': 5, 'church': 5, 'using': 5, 'airplane': 5, 'automatic': 5, 'happening': 5, 'florida': 5, 'ended': 5, 'status': 5, 'education': 5, 'stopped': 5, 'steel': 5, 'officially': 5, 'itself': 5, 'potus': 5, 'tonto': 5, 'false': 5, 'twins': 5, 'following': 5, 'zombie': 5, 'fighting': 5, 'planet': 5, 'beginning': 5, 'romance': 5, 'mining': 5, 'blog': 5, 'taste': 5, 'luck': 5, 'rule': 5, 'prepared': 5, 'economic': 5, 'tracks': 5, 'staff': 5, 'fingers': 5, 'eblokeve': 5, 'encmhz': 5, 'slightly': 5, 'sport': 5, 'jackson': 5, 'leads': 5, 'mido': 5, 'charged': 5, 'admits': 5, 'scheme': 5, 'somehow': 5, 'patrick': 5, 'driver': 5, 'robot': 5, 'defense': 5, 'piece': 5, 'dragon': 5, 'clip': 5, 'external': 5, 'electronic': 5, 'pathogens': 5, 'anthrax': 5, 'mishaps': 5, 'france': 5, 'tour': 5, 'covered': 5, 'calories': 5, 'fail': 5, 'grade': 5, 'spanish': 5, 'rare': 5, 'nose': 5, 'written': 5, 'apparently': 5, 'fine': 5, 'characters': 5, 'instagram': 5, 'called'

In [84]:
def predict(data):
    preds = []
    X = data.copy()
    for i in range(X.shape[0]):
        text = re.sub('[^a-z]+', ' ', str(X['text'][i]).lower()).split()
        is_true, is_false = 0, 0
        for word in text:
            try:
                is_true += vocab[1][word]
            except: 1

            try:
                is_false += vocab[0][word]
            except: 1
        preds.append(int(is_true >= is_false))
    return preds

In [85]:
X_test = df_train
y_test = X_test['target']
preds = predict(X_test)
print(accuracy_score(y_test, preds))

0.6667542361749639


In [86]:
df_test = pd.read_csv('test.csv')
ID = df_test['id']
preds = predict(df_test)
df_test['target'] = preds
for i in range(30):
    print(df_test['text'][i], df_test['target'][i])

Just happened a terrible car crash 0
Heard about #earthquake is different cities, stay safe everyone. 0
there is a forest fire at spot pond, geese are fleeing across the street, I cannot save them all 1
Apocalypse lighting. #Spokane #wildfires 0
Typhoon Soudelor kills 28 in China and Taiwan 1
We're shaking...It's an earthquake 1
They'd probably still show more life than Arsenal did yesterday, eh? EH? 0
Hey! How are you? 1
What a nice hat? 0
Fuck off! 0
No I don't like cold! 0
NOOOOOOOOO! Don't do that! 0
No don't tell me that! 0
What if?! 0
Awesome! 0
Birmingham Wholesale Market is ablaze BBC News - Fire breaks out at Birmingham's Wholesale Market http://t.co/irWqCEZWEU 1
@sunkxssedharry will you wear shorts for race ablaze ? 0
#PreviouslyOnDoyinTv: Toke MakinwaÛªs marriage crisis sets Nigerian Twitter ablaze... http://t.co/CMghxBa2XI 1
Check these out: http://t.co/rOI2NSmEJJ http://t.co/3Tj8ZjiN21 http://t.co/YDUiXEfIpE http://t.co/LxTjc87KLS #nsfw 1
PSA: IÛªm splitting my personali

In [25]:
out = pd.DataFrame({
    'id': ID,
    'target': preds
})
out.to_csv('submission.csv', index = None)
out.head(10)

,id,target
0,0,0
1,2,0
2,3,1
3,9,0
4,11,1
5,12,1
6,21,0
7,22,1
8,27,0
9,29,0
